In [1]:
import os, code, sys, time
import numpy as np
import tensorflow as tf
import chainer
import chainer.functions as F
from chainer import cuda
from random import randrange
from numpy import linalg as LA
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import NearestNeighbors
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
import utils

In [2]:
'''
Graph, neighbors ops
'''

class KNN():
    def __init__(self, X_in, K, shell_fraction=0.1):
        self.K = K
        self.L_box = 16 if X_in.shape[-2] == 16**3 else 32

        # Change shell_fraction for tuning the thickness of the shell to be replicated, must be in range 0-1
        # 0: no replication, 1: entire box is replicated to the 26 neighbouring boxes
        self.shell_fraction = shell_fraction
        self.dL = self.L_box * self.shell_fraction
        self.adjacency_list = self.get_adjacency_list_periodic_bc(X_in)

    def _get_status(self, coordinate):
        """
        Assign a status to each coordinate (of a particle position inside the box):
        1 if 0 < coord < dL, 2 if L- dL < coord < L, 0 otherwise

        PARAMS:
            coordinate(float)
        RETURNS:
            status(int). Either 0, 1, or 2
        """
        if coordinate < self.dL:
            return 1
        elif self.L_box - self.dL < coordinate < self.L_box:
            return 2
        else:
            return 0

    def _get_clone(self, particle, k, s):
        """
        Clone a particle otuside of the box.

        PARAMS:
            particle(np array). 6-dim particle position in phase space
            k(int). Index of dimension that needs to be projected outside of the box.
            s(int). Status, either 1 or 2. Determines where should be cloned.
        RETURNS:
            clone(np array). 6-dim cloned particle position in phase space.
        """
        clone = []
        for i in range(6):
            if i == k:
                if s == 1:
                    clone.append(particle[i] + self.L_box)
                elif s == 2:
                    clone.append(particle[i] - self.L_box)
            else:
                clone.append(particle[i])
        return np.array(clone)

    def get_adjacency_list_periodic_bc(self, X_in):
        """
        Map inner chunks to outer chunks
        """
        K = self.K
        mb_size, N, D = X_in.shape
        adj_list = np.zeros([mb_size, N, K], dtype=np.int32)

        for i in range(mb_size):
            ids_map = {}  # For this batch will map new_id to old_id of cloned particles
            new_X = [part for part in X_in[i]]  # Start off with original cube
            for j in range(N):
                status = [self._get_status(X_in[i, j, k]) for k in range(3)]
                if sum(status) == 0:  # Not in the shell --skip
                    continue
                else:
                    for k in range(3):
                        if status[k] > 0:
                            clone = self._get_clone(particle=X_in[i, j, :], k=k, s=status[k])
                            new_X.append(clone)
                            ids_map.update({len(new_X) - 1: j})
                            for kp in range(k + 1, 3):
                                if status[kp] > 0:
                                    bi_clone = self._get_clone(particle=clone, k=kp, s=status[kp])
                                    new_X.append(bi_clone)
                                    ids_map.update({len(new_X) - 1: j})
                                    for kpp in range(kp + 1, 3):
                                        if status[kpp] > 0:
                                            tri_clone = self._get_clone(particle=bi_clone, k=kpp, s=status[kpp])
                                            new_X.append(tri_clone)
                                            ids_map.update({len(new_X) - 1: j})
            new_X = np.array(new_X)
            graph_idx = kneighbors_graph(new_X[:, :3], K, include_self=True).indices
            graph_idx = graph_idx.reshape([-1, K])[:N, :]  # Only care about original box
            # Remap outbox neighbors to original ids
            for j in range(N):
                for k in range(K):
                    if graph_idx[j, k] > N - 1:  # If outside of the box
                        graph_idx[j, k] = ids_map.get(graph_idx[j, k])
            graph_idx = graph_idx + (N * i)  # offset idx for batches
            adj_list[i] = graph_idx
        return adj_list
    
    def __call__(self,):
        # just return copy of adjacency list
        alist = np.copy(self.adjacency_list)
        return alist

def graph_convolution(x, knn):
    alist = knn().flatten()

In [9]:
''' 
See whether indexing ops equivalent
chainer.functions.get_item
tensorflow.gather
'''
X = utils.load_datum(32, 0.6, normalize_data=True)

xbatch = X[:8] # (8, n_P, 6)

knn = KNN(xbatch, 14)

chx = chainer.Variable(xbatch)
tfx = tf.Variable(xbatch)

alist = knn()
aflat = alist.flatten()

chx_flat = F.reshape(chx, (-1,6))
chx_getitem = F.get_item(chx_flat, aflat)
chx = F.reshape(chx_getitem, (8, 32768, 14, 6))

tfx_flat = tf.reshape(tfx, (-1,6))
tfx_gather = tf.gather(tfx_flat, aflat)
tfx = tf.reshape(tfx_gather, (8, 32768, 14, 6))

In [32]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [35]:
np.all(tfx.eval() == chx.data)

True

In [ ]:

from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}


# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer



In [4]:
num_particles = 32
henorm_std = lambda x: np.sqrt(2.0 / x)
weight = tf.random_normal((6,8), stddev=henorm_std(6))

In [1]:
channels = [6, 8, 16, 32, 16, 8, 3, 8, 16, 32, 16, 8, 3]
learning_rate = 0.01
batch_size = 8
num_iters = 1000

In [6]:
'''
Graph model
'''
WTAG = 'W{}_{}'
BTAG = 'b{}'
params = {}

def left_mult(T1,T2):
    Tout = tf.einsum('ijl,lq->ijq',T1,T2)
    return Tout

def init_params(channels, pdict):
    """ initialize parameters for graph model
    Args:
        channels (list): layer sizes, 0th is input size, -1th output
        pdict (dict): dictionary that holds network parameters
    """
    for i in range(len(channels) - 1):
        ch_in, ch_out = channels[i], channels[i+1]
        # weight params
        w_x = tf.Variable(tf.random_normal([ch_in, ch_out], stddev=std))
        w_g = tf.Variable(tf.random_normal([ch_in, ch_out], stddev=std))
        # bias
        b = tf.Variable((np.ones((ch_out)) * 1e-6).astype(np.float32))
        # assign
        pdict[WTAG.format('x', i)] = w_x
        pdict[WTAG.format('g', i)] = w_g
        pdict[BTAG.format(i)] = b
    return pdict

def set_fwd(h, W):
    h_dims = tf.shape(h)
    

def graph_fwd(h, Wx, Wg, B, adjacency_list):
    # note SetLinear is different, mean isnt taken again on graph
    
    

# not sure it matters to make params dynamically because it's tensorflow... everything has to be hardcoded
def graph_net(x, adjacency_list):
    """ Graph network
    """
    knn = KNN(x)

{'Wg_0': <tf.Variable 'Variable_1:0' shape=(6, 8) dtype=float32_ref>,
 'Wg_1': <tf.Variable 'Variable_4:0' shape=(8, 16) dtype=float32_ref>,
 'Wg_10': <tf.Variable 'Variable_31:0' shape=(16, 8) dtype=float32_ref>,
 'Wg_11': <tf.Variable 'Variable_34:0' shape=(8, 3) dtype=float32_ref>,
 'Wg_2': <tf.Variable 'Variable_7:0' shape=(16, 32) dtype=float32_ref>,
 'Wg_3': <tf.Variable 'Variable_10:0' shape=(32, 16) dtype=float32_ref>,
 'Wg_4': <tf.Variable 'Variable_13:0' shape=(16, 8) dtype=float32_ref>,
 'Wg_5': <tf.Variable 'Variable_16:0' shape=(8, 3) dtype=float32_ref>,
 'Wg_6': <tf.Variable 'Variable_19:0' shape=(3, 8) dtype=float32_ref>,
 'Wg_7': <tf.Variable 'Variable_22:0' shape=(8, 16) dtype=float32_ref>,
 'Wg_8': <tf.Variable 'Variable_25:0' shape=(16, 32) dtype=float32_ref>,
 'Wg_9': <tf.Variable 'Variable_28:0' shape=(32, 16) dtype=float32_ref>,
 'Wx_0': <tf.Variable 'Variable:0' shape=(6, 8) dtype=float32_ref>,
 'Wx_1': <tf.Variable 'Variable_3:0' shape=(8, 16) dtype=float32_ref>

In [2]:
'''
Set array backend, seed
'''
rng_seed = 98765
use_gpu = True
np.random.seed(rng_seed)
print('rng_seed = {} '.format(rng_seed))

rng_seed = 98765 


In [3]:
'''
Dataset parameters
'''
num_particles = 16 # defaults 16**3
zX = 0.4
zY = 0.0
#zXb = 0.6
#zYb = 0.2
print('Using redshifts z{}, z{}, with {} particles'.format(zX,zY,num_particles**3))

Using redshifts z0.4, z0.0, with 4096 particles


In [5]:
'''
Load training dataset
'''
X,Y = load_data(zX, zY, num_particles)
X = normalize(X)
Y = normalize(Y)
#Xb, Yb = load_data(zXb, zYb, num_particles)
#Xb = normalize(Xb)
#Yb = normalize(Yb)
print('X.shape: {}'.format(X.shape))

X.shape: (1000, 4096, 6)


In [4]:
'''
Pre-processing: nearest-neighbors and sparse adjacency
'''

'''
SPARSE ADJACENCY MATRIX 
• scikit learn gives a CRS sparse adjacency for example, sparse TF takes COO. Use this for sparse X dense matmul in TF.
• also return adjacency lists and convert alist into index list to be used for generic normalizations (avg, max, etc)
'''

# returns adjacency lists based on NN in coordinate space
def adjacency_list(X_in,k):
    shape_in = X_in.shape
    X_out = np.zeros([shape_in[0],shape_in[1],k],dtype=np.int32)
    for b in range(shape_in[0]):
        X_out[b] = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([shape_in[1],k])
    return X_out

def get_adjacency_list(X_in,k):
    """ search for k nneighbors, and return offsetted indices in adjacency list
    
    Args:
        X_in: input data of shape (mb_size, N, 6)
        k: number of nearest neighbors
    """
    mb_size, N, D = X_in.shape
    X_out = np.zeros([mb_size, N, k],dtype=np.int32)
    for b in range(mb_size):
        # this returns indices of the nn
        graph_idx = kneighbors_graph(X_in[b,:,:3],k,include_self=True).indices.reshape([N,k]) + (N * b)
        X_out[b] = graph_idx
    return X_out

# adjacency list to proper index list for get_item
def alist_to_indexlist(alist):
    """ tiles batch indices to adjacency list for tf.gather
    """
    b, n, k = alist.shape
    #b = alist.shape[0] # batch size
    #n = alist.shape[1] # set size
    #k = alist.shape[2] # number of nn
    id1 = np.reshape(np.arange(b),[b,1])
    id1 = np.tile(id1,n*k).flatten()
    out = np.stack([id1,alist.flatten()],axis=1)
    return out

In [13]:
channels = [6, 8, 16, 32, 16, 8, 3, 8, 16, 32, 16, 8, 3] # 'bi',lowest loss for graph model with 0.01 8
#X_input = tf.placeholder(tf.float32, shape=[None,N_p,k_in], name='X_input')
x10 = X[:10]
layer1 = tf.layers.Dense(8)

AttributeError: module 'tensorflow' has no attribute 'tensor'

AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'